# Problem
Our problem is a classification problem, our target is whether a shot is goal or not.

# Where Can We Use This Model In Real World ?

We can use this model to analyze player performances, understand player habits on the field, determining which players to change in game or find out in advance whether a new player will be compatible with the team or not during transfer seasons.

# Imports

In [255]:
import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
import rpy2.robjects.packages as rpackages
from rpy2.robjects.conversion import localconverter
from imblearn.over_sampling import SMOTE
from collections import Counter
import warnings
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
import h2o
from h2o.automl import H2OAutoML
import pickle

In [256]:
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1)

<rpy2.rinterface_lib.sexp.NULLType object at 0x13faad3d0> [0]

# Getting Dataset Over R from Python

In [257]:
pandas2ri.activate()
ro.r('''
        library("worldfootballR")
        laliga <- load_understat_league_shots(league = "La liga")
     ''')
laliga = pandas2ri.rpy2py(ro.r['laliga'])
laliga.drop('league', axis=1, inplace=True)
data = laliga[(laliga['date'] > '2022-01-01') & (laliga['date'] < '2024-06-10')]

→ Data last updated 2024-05-30 18:34:46.012307882309 UTC


We are going to use La Liga dataset between 2023-01-01 and 2024-06-10.

# Data Manipulations and Fixes

We have a problem about NaN values and duplicate features, we are going to fix these problems by manipulating the data.

In [258]:
warnings.filterwarnings('ignore')

def fixDataNaN(df):
    with localconverter(ro.default_converter + pandas2ri.converter):
        df = ro.conversion.py2rpy(df)
pairs = [['x','X'],['y','Y'],['x_g','xG'],['h_a','home_away'],['shot_type','shotType'],['last_action','lastAction']]

def camel_case_columns(df):
    def camel_case(column_name):
        parts = column_name.split('_')
        return str(parts[0] + ''.join(x.title() for x in parts[1:]))
    
    new_columns = []
    for column in df.columns:
        if '_' in column:
            new_columns.append(camel_case(column))
        else:
            new_columns.append(str(column))
    
    df.columns = new_columns
    return df

def fixMergeColumns(dataList, pairs):
    for targetData in dataList:
        for pair in pairs:
            if pair[0] in targetData.columns and pair[1] in targetData.columns:
                targetData['{}'.format(pair[1])].fillna(targetData['{}'.format(pair[0])], inplace=True)
                targetData.drop(columns=['{}'.format(pair[0])], inplace=True)
        targetData = camel_case_columns(targetData)
        fixDataNaN(targetData)

fixMergeColumns([data], pairs)

We are going to train our model to predict whether the position ends up to a goal or not so we need to convert our goal and not goal situations to binary tags.

In [259]:
replacement_dict = {
    'Goal': 'Goal',
    'BlockedShot': 'No Goal',
    'MissedShots': 'No Goal',
    'SavedShot': 'No Goal',
    'ShotOnPost': 'No Goal',
    'OwnGoal': 'No Goal'
}

data['result'] = data['result'].map(replacement_dict)

# Exploring Data

In [260]:
data.head(2)

,id,minute,result,X,Y,xG,player,playerId,situation,season,shotType,matchId,homeTeam,awayTeam,homeGoals,awayGoals,date,playerAssisted,lastAction,homeAway
66772,451021.0,8.0,Goal,0.907,0.404,0.404479,Enes Ünal,6219.0,OpenPlay,2021.0,RightFoot,17323.0,Getafe,Real Madrid,1.0,0.0,2022-01-02 13:00:00,None,Tackle,h
66773,451022.0,13.0,No Goal,0.738,0.457,0.281229,Nemanja Maksimovic,6076.0,OpenPlay,2021.0,LeftFoot,17323.0,Getafe,Real Madrid,1.0,0.0,2022-01-02 13:00:00,None,Dispossessed,h


### Target Variable

Our target variable is "result", this feature represents whether a shot is a goal or not.

### Feature Variables

Our feature variables are going to help our model to learn and predict the target variable.

In [261]:
print(data.drop('result',axis=1).columns.tolist())

['id', 'minute', 'X', 'Y', 'xG', 'player', 'playerId', 'situation', 'season', 'shotType', 'matchId', 'homeTeam', 'awayTeam', 'homeGoals', 'awayGoals', 'date', 'playerAssisted', 'lastAction', 'homeAway']


# Imbalancedness

Our data has imbalancedness problem which might cause our model to learn features of majority target variables better than the minority target variables, it might cause inaccurate predictions.

In [262]:
print(data['result'].value_counts())

result
No Goal    20985
Goal        2379
Name: count, dtype: int64


In [263]:
X = data.drop('result', axis=1)
Y = data['result']

X = pd.get_dummies(X)

data = pd.concat([pd.DataFrame(X), pd.DataFrame(Y)], axis=1)

In [264]:
data_shuffled = data.sample(frac=1, random_state=42)
half_length = len(data_shuffled) // 2
df_half1 = data_shuffled.iloc[:half_length]
df_half2 = data_shuffled.iloc[half_length:]

data = df_half1
validation_data = df_half2

We are going to use SMOTE method for oversampling to fix the gap between minority and majority target variables. This method uses clustering methods to create new observations based on original ones.

In [265]:
Y = data['result']
x = data.drop('result', axis=1)
scaler = MinMaxScaler()
X = scaler.fit_transform(x)

print('Original dataset shape %s' % Counter(Y))
sm = SMOTE(random_state=42,n_jobs=-1)
x_res, y_res = sm.fit_resample(X, Y)
print('Resampled dataset shape %s' % Counter(y_res))

Original dataset shape Counter({'No Goal': 10478, 'Goal': 1204})
Resampled dataset shape Counter({'No Goal': 10478, 'Goal': 10478})


In [266]:
Y_validation = validation_data['result']
x_validation = validation_data.drop('result', axis=1)

X_validation = scaler.fit_transform(x_validation)

# Model

In [267]:
x_train, x_test, y_train, y_test = train_test_split(x_res, y_res, test_size=0.25, random_state=42, stratify=y_res)

We have found the best model to use is Random Forest after comparing Decision Tree, Logistic Regression,XGBoost and Random Forest.

In [268]:
model = RandomForestClassifier()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

5-Fold Cross Validation score:

In [269]:
print(cross_val_score(model, x_train, y_train, cv=5, scoring='accuracy').mean())

0.9611250899652607


### Train - Test Accuracy Comparison to Check Overfitting

Our model seems to have learned train set perfectly, we might have suspected of overfitting much more than the current situation if the test accuracies were bad but accuracy and balanced accuracy scores on train and test sets are very close to each other. Our model is good to go, but we are going to check if hyperparameter tuning or automl could improve our performance.

In [270]:
test_accuracy = accuracy_score(y_test, y_pred)
test_balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
test_confusion_matrix = confusion_matrix(y_test, y_pred)

train_accuracy = accuracy_score(y_train, model.predict(x_train))
train_balanced_accuracy = balanced_accuracy_score(y_train, model.predict(x_train))
train_confusion_matrix = confusion_matrix(y_train, model.predict(x_train))

In [271]:
print('Test Accuracy: {}\nTrain Accuracy: {}\n'.format(test_accuracy, train_accuracy))
print('Test Balanced Accuracy: {}\nTrain Balanced Accuracy: {}\n'.format(test_balanced_accuracy, train_balanced_accuracy))
print('Test Confusion Matrix: \n{}\n\nTrain Confusion Matrix: \n{}'.format(test_confusion_matrix, train_confusion_matrix))

Test Accuracy: 0.9622065279633518
Train Accuracy: 1.0

Test Balanced Accuracy: 0.9622076195972473
Train Balanced Accuracy: 1.0

Test Confusion Matrix: 
[[2506  114]
 [  84 2535]]

Train Confusion Matrix: 
[[7858    0]
 [   0 7859]]


### Hyperparameter Tuning

We are going to use the Halving Random Search method and 5-Fold Cross Validation together to find the best hyperparameters, evaluating our iterations by balanced accuracy scores.

The Halving Random Search method uses an elimination system as its base idea: the best of the two compared hyperparameter sets rises above on the leaderboard, and we get the best hyperparameters after the final round. Another pro of Halving Random Search is that it is much faster than Random Search.

In [272]:
param_grid = {
    'n_estimators': [int(x) for x in np.linspace(start=50, stop=5000, num=10)],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [int(x) for x in np.linspace(2, 50)],
    'min_samples_split': [int(x) for x in np.linspace(2, 5)],
    'min_samples_leaf': [int(x) for x in np.linspace(2, 5)],
    'bootstrap': [True, False],
}

halving = HalvingRandomSearchCV(model, param_grid, factor=3, resource='n_samples', max_resources=1000, random_state=42, verbose=0,scoring='balanced_accuracy', n_jobs=-1)
halving.fit(x_train, y_train)
print("Best Params:{}/nBest Balanced Accuracy:{}".format(halving.best_params_,halving.best_score_))

/Users/sezaiufukoral/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:395: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/Users/sezaiufukoral/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2466: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/sezaiufukoral/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:395: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/Users/sezaiufukoral/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2466: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/sez

Best Params:{'n_estimators': 4450, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 32, 'bootstrap': False}/nBest Balanced Accuracy:0.8637369061968074


We did not get what we wanted from hyperparameter tuning, our balanced accuracy decreased and it might not be ideal to consume more time to random searching, so let's see how AutoML is going to work.

In [273]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,4 hours 19 mins
H2O_cluster_timezone:,Europe/Istanbul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.2
H2O_cluster_version_age:,27 days
H2O_cluster_name:,H2O_from_python_sezaiufukoral_x1b2rc
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,808 Mb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [274]:
hf = h2o.H2OFrame(pd.concat([pd.DataFrame(x_res), pd.DataFrame(y_res)], axis=1))

train_hf, test_hf = hf.split_frame(ratios=[0.75], seed = 1)

### AutoML

We are going to use the H2O library because of its ease of use. We are going to set the maximum runtime to 300 seconds so our process won't consume too much time. AutoML is going to use 5-Fold Cross Validation by default and compare balanced accuracies and more metrics to choose the best model for our data.

In [275]:
aml = H2OAutoML(max_models = 20,
                balance_classes=True,
		seed =1,max_runtime_secs=600,verbosity='none')

aml.train(training_frame = train_hf, y = 'result')

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_1_AutoML_6_20240609_185301


Model Summary: 
    number_of_trees    number_of_nternal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  -------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    165                165                        116235                 10           15           14.5636       24            71            50.8545

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.0127313504909891
RMSE: 0.11283328627222156
LogLoss: 0.05919630477782149
Mean Per-Class Error: 0.011555413615394703
AUC: 0.9994814824380508
AUCPR: 0.9994816985734739
Gini: 0.9989629648761016

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5385781520815164
         Goal    No Goal    Error    Rate
-------  ------  ---------  -------  ---------------
Goal     5401    94         0.0171   (94.0/5495.0)
No Goal  33      5463       0.006    (33.0/5496.0)
Total    5434    5557       0.0116   (127.0/10991.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.538578     0.98851   205
max f2                       0.447412     0.992647  226
max f0point5                 0.729062     0.989909  155
max accuracy                 0.538578     0.988445  205
max precision                0.999307     1         0
max recall                   0.187021     1         291
max specificity              0.999307     1         0
max absolute_mcc             0.538578     0.97695   205
max min_per_class_accuracy   0.607458     0.987261  188
max mean_per_class_accuracy  0.538578     0.988445  205
max tns                      0.999307     5495      0
max fns                      0.999307     5363      0
max fps                      0.000299055  5495      399
max tps                      0.187021     5496      291
max tnr                      0.999307     1         0
max fnr                      0.999307     0.975801  0
max fpr                      0.000299055  1         399
max tpr                      0.187021     1         291

Gains/Lift Table: Avg response rate: 50,00 %, avg score: 49,96 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100082                   0.999104           1.99982   1.99982            1                0.999364     1                           0.999364            0.0200146       0.0200146                  99.9818   99.9818            0.0200146
2        0.0200164                   0.998678           1.99982   1.99982            1                0.998889     1                           0.999127            0.0200146       0.0400291                  99.9818   99.9818            0.0400291
3        0.0300246                   0.998332           1.99982   1.99982            1                0.998519     1                           0.998924            0.0200146       0.0600437                  99.9818   99.9818            0.0600437
4        0.0400328                   0.998056           1.99982   1.99982            1                0.998198     1                           0.998743            0.0200146       0.0800582                  99.9818   99.9818            0.0800582
5        0.0500409                   0.997663           1.99982   1.99982            1   

AutoML found the best model to be Gradient Boosting Machines. This algorithm is based on the idea of creating an ensemble of weak learners, typically decision trees, in a sequential manner. Each new model attempts to correct the errors of the previous models.

Our new model's performance is almost the same as vanilla Random Forest. However, AutoML checked more metrics to validate that this is the right model for our data. Both models are black box models, so it does not affect us which one we choose for the sake of interpretability. We can trust more in the one that AutoML found for its generalizability because it checked more metrics when building the model. So, we are good to go now.

In [276]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GBM_1_AutoML_6_20240609_185301,0.982868,0.157277,0.979261,0.0540629,0.212556,0.0451799
XGBoost_1_AutoML_6_20240609_185301,0.980755,0.165348,0.976219,0.0552033,0.216288,0.0467807
XGBoost_2_AutoML_6_20240609_185301,0.979706,0.164984,0.971498,0.0564989,0.213872,0.0457413
GBM_2_AutoML_6_20240609_185301,0.979438,0.184273,0.974896,0.0649669,0.22866,0.0522856
DRF_1_AutoML_6_20240609_185301,0.970245,0.371753,0.969912,0.0815848,0.323399,0.104587
GLM_1_AutoML_6_20240609_185301,0.936036,0.316161,0.949772,0.116551,0.303505,0.092115


In [277]:
warnings.filterwarnings('ignore')
preds = aml.leader.predict(test_hf)

y_test_gbm = test_hf['result'].as_data_frame().values.flatten()
y_pred_gbm = preds['predict'].as_data_frame().values.flatten()

balanced_acc = balanced_accuracy_score(y_test_gbm, y_pred_gbm)
print("Balanced Accuracy Score: ", balanced_acc)

Balanced Accuracy Score:  0.9476668326715598


# Validation of the Model

We are going to test our model with a new, unseen data.

In [280]:
warnings.filterwarnings('ignore')

validation_x_hf = h2o.H2OFrame(pd.DataFrame(X_validation))
validation_y_hf = h2o.H2OFrame(pd.DataFrame(Y_validation))

                               
preds_validation = aml.leader.predict(validation_x_hf)

y_validation_test_gbm = validation_y_hf['result'].as_data_frame().values.flatten()
y_validation_pred_gbm = preds_validation['predict'].as_data_frame().values.flatten()

balanced_acc = balanced_accuracy_score(y_validation_test_gbm, y_validation_pred_gbm)
print("Balanced Accuracy Score: ", balanced_acc)

print(confusion_matrix(y_validation_test_gbm, y_validation_pred_gbm))

Balanced Accuracy Score:  0.6376103874013069
[[  350   825]
 [  238 10269]]


In [279]:
y_validation_pred = model.predict(X_validation)

print(confusion_matrix(Y_validation, y_validation_pred))

print(balanced_accuracy_score(Y_validation, y_validation_pred))

[[  385   790]
 [  330 10177]]
0.6481259707307591
